In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize

### Store CSV into DataFrame

In [ ]:
csv_file = "Resources/Unempoyment_rate.csv"
Unempoyment_rate_df = pd.read_csv(csv_file)
Unempoyment_rate_df.head()

## Store JSON data into a DataFrame

In [ ]:
# json_file = "Resources/Unempoyment_yearlry_rate.json"
# # json_normalize(json_file)
# # Unempoyment_yearlry_rate_df = pd.read_json(json_file)
# Unempoyment_yearlry_rate_df=pd.read_json(json_file, orient='index')
# # Unempoyment_yearlry_rate_df=pd.read_json(json_file, orient='split')
# Unempoyment_yearlry_rate_df.head()

In [ ]:
json_file = "Resources/Unempoyment_yearlry_rate.json"
print(json_file)

In [ ]:
filepath = json_file
with open(filepath) as jsonfile:
    json_norm = json.load(jsonfile)
print(json_norm)

In [6]:
json_normalize(json_norm)

,1990.Annual.Alabama.Unemployment Rate.Autauga County,1990.Annual.Alabama.Unemployment Rate.Baldwin County,1990.Annual.Alabama.Unemployment Rate.Barbour County,1990.Annual.Alabama.Unemployment Rate.Bibb County,1990.Annual.Alabama.Unemployment Rate.Blount County,1990.Annual.Alabama.Unemployment Rate.Bullock County,1990.Annual.Alabama.Unemployment Rate.Butler County,1990.Annual.Alabama.Unemployment Rate.Calhoun County,1990.Annual.Alabama.Unemployment Rate.Chambers County,1990.Annual.Alabama.Unemployment Rate.Cherokee County,...,2016.September.Wyoming.Unemployment Rate.Niobrara County,2016.September.Wyoming.Unemployment Rate.Park County,2016.September.Wyoming.Unemployment Rate.Platte County,2016.September.Wyoming.Unemployment Rate.Sheridan County,2016.September.Wyoming.Unemployment Rate.Sublette County,2016.September.Wyoming.Unemployment Rate.Sweetwater County,2016.September.Wyoming.Unemployment Rate.Teton County,2016.September.Wyoming.Unemployment Rate.Uinta County,2016.September.Wyoming.Unemployment Rate.Washakie County,2016.September.Wyoming.Unemployment Rate.Weston County
0,6.5,5.2,7.8,9.1,6.3,12.4,12.8,7.1,7.2,9.9,...,3.0,3.6,4.0,3.8,5.4,5.5,2.0,5.1,3.8,4.7


In [7]:
print(json.dumps(json_norm, indent=4))


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
json_normed=json_normalize(json_norm,)

In [9]:
Unempoyment_yearlry_rate_df=Unempoyment_yearlry_rate_df.reset_index()
# Unempoyment_yearlry_rate_df=Unempoyment_yearlry_rate_df.pivot(index='Year',columns='months', values='rate')
# Unempoyment_yearlry_rate_df.head()

NameError: name 'Unempoyment_yearlry_rate_df' is not defined

### Create new data with select columns

### Clean DataFrame

### Connect to local database

### Check for tables

### Use pandas to load csv converted DataFrame into database

### Use pandas to load json converted DataFrame into database